# Stable Matching Pair Research
Implementation of [Stable Roomates](http://www.dcs.gla.ac.uk/~pat/jchoco/roommates/papers/Comp_sdarticle.pdf) for [Pair Research](http://pairresearch.io/). 

input from pair research: [(0, 1, 93), (0, 2, -20), (0, 3, 2), (1, 2, -13), (1, 3, 10), (2, 3, 80)]
- Single edge between two users with some weight for said edge

In [1]:
import random
from copy import deepcopy


def stable_roommates(preferences):
    """
    Runs complete algorithm returns a stable matching, if exists.

    Input:
        preferences (matrix, list of lists): n-by-m preference matrix containing preferences for each person.
            m = n - 1, so each person has rated all other people.

    Return:
        (list of tuples): stable matching, if exists. Otherwise, None.
    """
    # create a preference lookup table
    # person_number : [list of preferences]
    preferences_dict = {str(x + 1): [str(y) for y in preferences[x]] for x in xrange(len(preferences))}

    # create a dict of dicts holding index of each person ranked
    # person number : {person : rank_index }
    ranks = {index: dict(zip(value, xrange(len(value)))) for index, value in preferences_dict.iteritems()}

    # phase 1: initial proposal
    p1_holds = phase_1(preferences_dict, ranks)

    # if anyone does not have a hold, stable matching is not possible
    for hold in p1_holds:
        if p1_holds[hold] is None:
            print 'Stable matching is not possible. Failed at Phase 1: not everyone was proposed to.'
            return None

    # phase 1: reduction
    p1_reduced_preferences = phase_1_reduce(preferences_dict, ranks, p1_holds)

    # phase 2: find an all-or-nothing cycle
    cycle = find_all_or_nothing_cycle(p1_reduced_preferences, ranks, p1_holds)

    # if cycle with more than size 3 does not exist, no stable matching exists
    if cycle is None:
        print 'Stable matching is not possible. Failed at Phase 2: could not find an all-or-nothing cycle.'
        return None
    elif cycle is not None and len(cycle) == 3:
        print 'Stable matching is not possible. Failed at Phase 2: could not find an all-or-nothing cycle len > 3.'
        return None

    # phase 2: reduction
    final_holds = phase_2_reduce(p1_reduced_preferences, ranks, cycle)

    # check if holds are not empty
    if final_holds is not None:
        return final_holds
    else:
        print 'Stable matching is not possible. Failed at Phase 2 reduction.'
        return None


def phase_1(preferences, ranks, curr_holds=None):
    """
    Performs first phase of matching by doing round robin proposals until stopping condition (i) or (ii) is met:
         (i) each person is holding a proposal
        (ii) one person is rejected by everyone

    Input:
        preferences (dict of list of strings): dict of ordered preference lists {person : [ordered list]}
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person
        curr_holds (dict, optional): dict of persons with current holds

    Return:
        (dict): holds after condition (i) or (ii) is met.
    """
    people = preferences.keys()

    # placeholder for holds
    holds = {person: None for person in people}

    # during phase 1, no holds exist. initialize curr_holds to 0 (all people are > 0)
    if curr_holds is None:
        curr_holds = {person: 0 for person in people}

    # randomize ordering of proposal
    random.shuffle(people)

    # track people who are already proposed to
    proposed_set = set()

    # begin proposing
    for person in people:
        proposer = person

        # proposal step
        while True:
            # find proposer someone to propose to, in order of proposer's preference list
            while curr_holds[proposer] < len(preferences[proposer]):
                # find proposee given proposer's preferences
                proposee = preferences[proposer][curr_holds[proposer]]
                curr_holds[proposer] += 1

                # find who proposee is holding, if any
                proposee_hold = holds[proposee]

                # stop searching if proposee doesn't hold anyone or ranks proposer higher than curr hold
                if proposee_hold is None or ranks[proposee][proposer] < ranks[proposee][proposee_hold]:
                    # proposee holds proposer's choice
                    holds[proposee] = proposer
                    break

            # check if proposee has already been proposed to
            if proposee not in proposed_set:
                break

            # if proposee is proposed to, reject proposee_hold and continue proposal with them
            proposer = proposee_hold

        # successful proposal
        proposed_set.add(proposee)

    # final holds from phase 1
    return holds


def phase_1_reduce(preferences, ranks, holds):
    """
    Performs a reduction on preferences based on phase 1 proposals, and the following:
        Preference list for y who holds proposal x can be reduced by deleting
             (i) all those to whom y prefers x
            (ii) all those who hold a proposal from a person they prefer to y

    Input:
        preferences (dict of list of strings): dict of ordered preference lists {person : [ordered list]}
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person
        holds (dict): dict of persons with current holds

    Return:
        (dict of list of strings): reduced preference list such that:
            (iii) y is the first on x's list and last on y's
             (iv) b appears on a's list iff a appears on b's
    """
    # create output preferences
    reduced_preferences = deepcopy(preferences)

    # loop though each hold
    for proposee in holds:
        proposer = holds[proposee]

        # loop though all of person's preferences
        i = 0
        while i < len(reduced_preferences[proposee]):
            # fetch proposee's preferences
            curr_proposee_preference = reduced_preferences[proposee][i]

            # proposee should only hold preferences equal and higher to proposer (i)
            if curr_proposee_preference == proposer:
                reduced_preferences[proposee] = reduced_preferences[proposee][:(i + 1)]
            # delete all people who hold a proposal from someone they prefer to the proposee (ii)
            elif ranks[curr_proposee_preference][holds[curr_proposee_preference]] < \
                    ranks[curr_proposee_preference][proposee]:
                reduced_preferences[proposee].pop(i)
                continue

            # continue to preference list
            i += 1

    return reduced_preferences


def find_all_or_nothing_cycle(preferences, ranks, holds):
    """
    Finds an all-or-nothing cycle in reduced preferences, if exists.

    Input:
        preferences (dict of list of strings): dict of ordered preference lists {person : [ordered list]}
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person
        holds (dict): dict of persons with current holds

    Return:
        (list): cycle of users
    """
    # start with two individuals, p and q
    p = []
    q = []

    # find a person with > 1 preference left
    curr = None
    for person in preferences:
        if len(preferences[person]) > 1:
            curr = person
            break

    # if no person can be found, no cycle exists
    if curr is None:
        return None

    # create cycle
    while curr not in p:
        # q_i = second person in p_i's list
        q += [preferences[curr][1]]

        # p_{i + 1} = q_i's last person
        p += [curr]
        curr = preferences[q[-1]][-1]

    cycle = p[p.index(curr):]

    return cycle


def phase_2_reduce(preferences, ranks, cycle):
    """
    Performs a reduction on found all-or-nothing cycles.

    Input:
        preferences (dict of list of strings): dict of ordered preference lists {person : [ordered list]}
        ranks (dict of dict of ranking index): dict of persons with dicts indicating rank of each other person
        cycle (list): all-or-nothing cycle

    Return:
        (dict): holds after sequential reductions, or None if no matching can be found.
    """
    # continue while a cycle exists
    curr_cycle = deepcopy(cycle)
    curr_holds = None
    p2_preferences = deepcopy(preferences)

    while curr_cycle is not None:
        curr_preferences = {}

        for person in preferences:
            if person in curr_cycle:
                curr_preferences[person] = 1
            else:
                curr_preferences[person] = 0

        curr_holds = phase_1(p2_preferences, ranks, curr_preferences)
        p2_preferences = phase_1_reduce(p2_preferences, ranks, curr_holds)

        curr_cycle = find_all_or_nothing_cycle(p2_preferences, ranks, curr_holds)

    return curr_holds


In [2]:
paper_matching = [[4, 6, 2, 5, 3],
                  [6, 3, 5, 1, 4],
                  [4, 5, 1, 6, 2],
                  [2, 6, 5, 1, 3],
                  [4, 2, 3, 6, 1],
                  [5, 1, 4, 2, 3]]

paper_no_matching = [[2, 6, 4, 3, 5],
                     [3, 5, 1, 6, 4],
                     [1, 6, 2, 5, 4],
                     [5, 2, 3, 6, 1],
                     [6, 1, 3, 4, 2],
                     [4, 2, 5, 1, 3]]

matching = [[8, 14, 9, 16, 2, 3, 7, 10],
            [1, 5, 10, 20, 4, 13, 18, 11],
            [8, 5, 11, 16, 10, 15, 18, 13, 1, 4, 17],
            [20, 14, 19, 7, 6, 2, 17, 11, 3, 8],
            [20, 17, 2, 16, 7, 8, 9, 14, 15, 3, 10],
            [16, 4, 15, 14, 12, 8, 9],
            [5, 14, 4, 12, 19, 11, 1, 17, 8, 15, 10],
            [6, 4, 10, 1, 7, 11, 13, 3, 17, 5],
            [17, 1, 6, 10, 16, 5],
            [3, 18, 7, 8, 12, 5, 9, 1, 2, 19, 17],
            [2, 3, 8, 13, 7, 15, 19, 4],
            [10, 13, 17, 6, 7, 19, 16, 18],
            [8, 20, 11, 12, 2, 3, 15, 14],
            [6, 17, 13, 7, 19, 5, 4, 18, 1],
            [20, 13, 5, 7, 6, 3, 19, 18, 11],
            [19, 9, 3, 5, 1, 12, 6],
            [4, 5, 12, 10, 14, 3, 7, 9, 8, 20],
            [10, 20, 15, 3, 19, 14, 2, 12],
            [4, 15, 14, 10, 7, 12, 16, 18, 11],
            [18, 13, 5, 17, 15, 2, 4]]

no_matching = [[2, 3, 4],
               [3, 1, 4],
               [1, 2, 4],
               [1, 2, 3]]

In [3]:
paper_matching_res = stable_roommates(paper_matching)
zip(paper_matching_res.keys(), paper_matching_res.values())

[('1', '6'), ('3', '2'), ('2', '3'), ('5', '4'), ('4', '5'), ('6', '1')]

In [4]:
stable_roommates(paper_no_matching)

Stable matching is not possible. Failed at Phase 2: could not find an all-or-nothing cycle len > 3.


In [5]:
matching_res = stable_roommates(matching)
zip(matching_res.keys(), matching_res.values())

[('11', '2'),
 ('10', '3'),
 ('13', '12'),
 ('12', '13'),
 ('15', '6'),
 ('14', '7'),
 ('17', '5'),
 ('16', '9'),
 ('19', '4'),
 ('18', '20'),
 ('1', '8'),
 ('3', '10'),
 ('2', '11'),
 ('5', '17'),
 ('4', '19'),
 ('7', '14'),
 ('6', '15'),
 ('9', '16'),
 ('20', '18'),
 ('8', '1')]

In [6]:
stable_roommates(no_matching)

Stable matching is not possible. Failed at Phase 2: could not find an all-or-nothing cycle.


## old

In [ ]:
class Person(object):
    """
    Person object with preferences. 
    
    Attributes:
        name: name of person, referred to by number
        preferences: ordered list of strings detailing pairing preferences. 
            Given n users in a pool, len(preferences) = n - 1 as each person ranks every other person in the pool.
        preference_index: dict containing index of preference for fast lookup of ranking.
    """
    def __init__(self, name, preferences):
        """
        Returns a Person object with attributes initialized.
        
        Input:
            preferences (list of strings): ordered list of strings detailed pairing preferences
                ex w/4 users: [[]]
        """
        self.name = str(name)
        self.preferences = preferences + name
        self.preference_index = {}
        
        # generate preference index
        for i in xrange(len(preferences)): 
            self.preference_index[preferences[i]] = str(i)
    
    def leftmost(self): 
        """
        Gets next choice for person to propose to.
        """
        return self.preferences[0]
    
    def rightmost(self): 
        """
        Gets next choice for person to propose to.
        """
        return self.preferences[0]           
    
class ResearchPool(object):
    """
    Pool of Persons to match over.
    
    Attributes:
        persons: list of people in pool
        person_index: dict containing name:Person mapping for fast lookup of people
        proposal_set: set of proposals made
    """
    def __init__(self, preference_matrix):
        """
        Returns a ResearchPool object with attributes intitialized.

        Input:
            preference_matrix (matrix, list of lists): n-by-m matrix of n people with m = n - 1 preferences each.
                preferences should be 1-indexed as each person will be 1-indexed.
        """
        self.persons = [Person(x + 1, preference_matrix[x]) for x in xrange(len(preference_matrix))]
        self.person_index = {}
        self.proposal_set = set()

        # check for an odd number of people
        if (len(self.persons) % 2) != 0: 
            self.persons.append(Person(len(self.persons), [x for x in xrange(len(self.persons[0]))]))
            
        # create person_index
        for i in self.persons: 
            self.person_index[i.name] = i
            
    def clear_proposal_set(self):
        """
        Empties the proposal_set attribute.
        """
        self.proposal_set = set()
    
    def get_person(self, person_name):
        """
        Get and return a Person in the research pool, by name
        
        Input: 
            person_name (string): name of person.
        
        Return: 
            (Person): Person object with the name
        """
        return self.persons[self.person_index[person_name]]

    def __str__(self):
        """
        Custom print for research pool in the format user | preference list.
        """
        output = ""
        for i in xrange(len(self.persons)):
            preference_string = " ".join([str(x) for x in self.persons[i].preferences])
            output += "{} | {}".format(i + 1, preference_string)
            output += "\n"
            
        return output

In [ ]:
def create_pool(preference_matrix):
    """
    Creates a ResearchPool that stable matching can be used upon. 
    
    Input: 
        preference_matrix (matrix, list of lists): and n-by-m matrix of n people with m = n - 1 preferences each
    
    Return: 
        (ResearchPool): a ResearchPool object initialized with the preference_matrix
    """
    # validate preference matrix size
    n = len(preference_matrix)
    m = n - 1

    for i in preference_matrix:
        if len(i) is not m: 
            raise TypeError("Preference matrix must be n-by-m with m = n - 1 for all n.")
    
    return ResearchPool(preference_matrix)

pool = create_pool([[4, 6, 2, 5, 3],
                    [6, 3, 5, 1, 4],
                    [4, 5, 1, 6, 2],
                    [2, 6, 5, 1, 3],
                    [4, 2, 3, 6, 1],
                    [5, 1, 4, 2, 3]])

print pool